# Deep-Live-Cam Complete Features
All features working properly

In [ ]:
# Setup
!apt update -qq && apt install -y ffmpeg
!git clone https://github.com/Mayank-kanojiya/liveswap.git
%cd liveswap
!pip install -q opencv-python onnxruntime-gpu torch torchvision insightface
!pip install -q customtkinter pillow psutil protobuf cv2-enumerate-cameras
!pip install -q git+https://github.com/xinntao/BasicSR.git@master
!pip install -q git+https://github.com/TencentARC/GFPGAN.git@master

In [ ]:
# Download models
import os
os.makedirs('models', exist_ok=True)
!wget -q -O models/GFPGANv1.4.pth https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth
!wget -q -O models/inswapper_128_fp16.onnx https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx

In [ ]:
# Complete features
import gradio as gr
import cv2
import tempfile
import sys
import os
sys.path.append('.')

import modules.globals
modules.globals.headless = True
modules.globals.execution_providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']

from modules.processors.frame.face_swapper import get_face_swapper, process_frame
from modules.face_analyser import get_one_face
from modules.utilities import is_video, extract_frames, create_video, restore_audio, get_temp_frame_paths, create_temp, clean_temp

face_swapper = get_face_swapper()

def complete_swap(source_image, target_media, mouth_mask, many_faces, keep_fps, keep_audio, face_enhancer):
    if source_image is None or target_media is None:
        return None, "Upload both files"
    
    try:
        modules.globals.mouth_mask = mouth_mask
        modules.globals.many_faces = many_faces
        modules.globals.keep_fps = keep_fps
        modules.globals.keep_audio = keep_audio
        
        processors = ['face_swapper']
        if face_enhancer:
            processors.append('face_enhancer')
        modules.globals.frame_processors = processors
        
        source_img = cv2.imread(source_image)
        source_face = get_one_face(source_img)
        
        if is_video(target_media.name):
            output_path = tempfile.mktemp(suffix='.mp4')
            create_temp(target_media.name)
            extract_frames(target_media.name)
            
            temp_frame_paths = get_temp_frame_paths(target_media.name)
            
            for i, frame_path in enumerate(temp_frame_paths):
                frame = cv2.imread(frame_path)
                result_frame = process_frame(source_face, frame)
                cv2.imwrite(frame_path, result_frame)
                
                if i % 30 == 0:
                    print(f"Processed {i+1}/{len(temp_frame_paths)} frames")
            
            create_video(target_media.name)
            if keep_audio:
                restore_audio(target_media.name, output_path)
            clean_temp(target_media.name)
            
            return output_path, "Video completed!"
        else:
            output_path = tempfile.mktemp(suffix='.jpg')
            target_img = cv2.imread(target_media.name)
            result_frame = process_frame(source_face, target_img)
            cv2.imwrite(output_path, result_frame)
            return output_path, "Image completed!"
        
    except Exception as e:
        return None, f"Error: {str(e)}"

print("Ready!")

In [ ]:
# Interface
with gr.Blocks() as demo:
    gr.Markdown("# Deep-Live-Cam Complete")
    
    with gr.Row():
        with gr.Column():
            source_image = gr.Image(label="Source Face", type="filepath")
            target_media = gr.File(label="Target Image/Video")
            
            mouth_mask = gr.Checkbox(label="Mouth Mask (eating preservation)", value=False)
            many_faces = gr.Checkbox(label="Process All Faces", value=False)
            keep_fps = gr.Checkbox(label="Keep Original FPS", value=True)
            keep_audio = gr.Checkbox(label="Keep Audio", value=True)
            face_enhancer = gr.Checkbox(label="Face Enhancement", value=False)
            
            process_btn = gr.Button("Process", variant="primary")
            
        with gr.Column():
            result = gr.File(label="Result")
            status = gr.Textbox(label="Status")
    
    process_btn.click(
        complete_swap,
        [source_image, target_media, mouth_mask, many_faces, keep_fps, keep_audio, face_enhancer],
        [result, status]
    )

demo.launch(share=True)